In [1]:
import requests
from bs4 import BeautifulSoup, element

import pandas as pd
import numpy as np

import src.scraping as scrap

In [2]:
jobs = scrap.main("data scientist", numb_pages=20)

100%|██████████| 284/284 [01:54<00:00,  2.49it/s]


In [8]:
len([j.title for _, j in jobs.items() if j.title is not None])

265

In [2]:
set([1]).add(set([2, 3]))

TypeError: unhashable type: 'set'

In [85]:
page = requests.get("https://uk.indeed.com/viewjob?jk=6ed3255bc10ff0dd")
soup = BeautifulSoup(page.content, "html.parser")

In [72]:
soup.find("h1", {"class": "jobsearch-JobInfoHeader-title"}).text

'Research Data Scientist'

In [73]:
soup.find("div", {"class" : "jobsearch-CompanyReview--heading"}).text

'University of Oxford'

In [74]:
float(soup.find("meta", {"itemprop" : "ratingValue"})["content"])

4.2

In [75]:
soup.find("meta", {"itemprop" : "ratingCount"})["content"]

'109'

In [77]:
soup.find("span", {"class":"attribute_snippet"}).text

AttributeError: 'NoneType' object has no attribute 'get'

In [70]:
soup.find("span", {"class":"jobsearch-JobMetadataHeader-item"}).text.replace(" -  ", "").split(", ")

['Full-time', 'Temporary']

In [83]:
soup.find("div", {"id":"jobDescriptionText"}).text

'Two part time roles 0.4 FTE, 15 hours per week\nFixed-term for 12 months\n\nWe are looking for outstanding candidates to fill two part-time research data scientist positions. These candidates will work closely with the FOMI academic team on data-intensive research projects. This is an excellent opportunity to gain academic research experience in business/marketing/technology, to learn from one of the world’s leading groups of academic researchers in marketing, and to work on research projects in collaboration with FOMI industry partners that will have significant impacts on their businesses.\n\nThe ideal candidate for these roles will have practical experience with quantitative/empirical modelling techniques, working with large datasets, and have a background in one or more fields such as statistics, machine learning, econometrics, applied computer science, or quantitative/computational social science. Experience with data science work on research projects (either in academia or in co

In [46]:
len(soup.find("div", {"id":"jobDescriptionText"}).text)

856

In [78]:
type(soup.find("div", {"id":"jobDescriptionText"}))

bs4.element.Tag

In [88]:
from dataclasses import dataclass

from bs4 import BeautifulSoup

@dataclass
class JobPost:
    job_id: str
    title: str
    company: str
    salary: str
    rating_value: float
    rating_count: int
    contract_type: list[str]
    description: str

    @classmethod
    def from_job_soup(cls, job_soup: BeautifulSoup, job_id: str):
        title = cls.text_if_not_none(job_soup.find("h1",
                                                   {"class": "jobsearch-JobInfoHeader-title"}))
        company = cls.text_if_not_none(job_soup.find("div",
                                                     {"class" : "jobsearch-CompanyReview--heading"}))
        salary = cls.text_if_not_none(job_soup.find("span",
                                                    {"class":"attribute_snippet"}))
        rating_value = float(cls.text_if_not_none(job_soup.find("meta",
                                                          {"itemprop" : "ratingValue"}), as_content=True))
        rating_count = int(cls.text_if_not_none(job_soup.find("meta",
                                                              {"itemprop" : "ratingCount"}), as_content=True))
        contract_type = cls.text_if_not_none(job_soup.find("span",
                                                           {"class":"jobsearch-JobMetadataHeader-item"})).replace(" -  ", "").split(", ")
        contract_type = [c for c in contract_type if len(c) > 3]
        description = cls.text_if_not_none(soup.find("div", {"id":"jobDescriptionText"}))
        return cls(job_id=job_id,
                   title=title,
                   company=company,
                   salary=salary,
                   rating_value=rating_value,
                   rating_count=rating_count,
                   contract_type=contract_type,
                   description=description
                   )
        
    @staticmethod
    def text_if_not_none(soup_search: element.Tag | None, as_content: bool = False):
        if soup_search is not None:
            if as_content:
                return soup_search["content"]
            return soup_search.text
        return None
        

In [90]:
post = JobPost.from_job_soup(soup, 12)

In [94]:
post.__dict__

{'job_id': 12,
 'title': 'Research Data Scientist',
 'company': 'University of Oxford',
 'salary': None,
 'rating_value': 4.2,
 'rating_count': 109,
 'contract_type': ['Part-time', 'Fixed term'],
 'description': 'Two part time roles 0.4 FTE, 15 hours per week\nFixed-term for 12 months\n\nWe are looking for outstanding candidates to fill two part-time research data scientist positions. These candidates will work closely with the FOMI academic team on data-intensive research projects. This is an excellent opportunity to gain academic research experience in business/marketing/technology, to learn from one of the world’s leading groups of academic researchers in marketing, and to work on research projects in collaboration with FOMI industry partners that will have significant impacts on their businesses.\n\nThe ideal candidate for these roles will have practical experience with quantitative/empirical modelling techniques, working with large datasets, and have a background in one or more fi